In [1]:

import panhumanpy as ph
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import pooch
from scipy.sparse import csr_matrix
from scipy.io import mmwrite
import re
import anndata
import scanpy.external as sce
import scvi
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import gseapy as gp
from gseapy.plot import dotplot
import math

2025-09-04 08:20:27.135368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-04 08:20:27.345080: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-04 08:20:27.404104: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-04 08:20:27.856880: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-04 08:20:30.478366: W tensorflow/comp

In [3]:
KPMP_SN_raw=sc.read('/scratch/yongqunh_root/yongqunh0/wruopeng/HuBMAP_KPMP/kpmp-sn-raw-rnaseq.h5ad')
HuBMAP_LK_raw=sc.read('/scratch/yongqunh_root/yongqunh0/wruopeng/HuBMAP_KPMP/LK_raw_updated_2025_june.h5ad')
HuBMAP_RK_raw=sc.read('/scratch/yongqunh_root/yongqunh0/wruopeng/HuBMAP_KPMP/RK_raw_updated_2025_june.h5ad')

In [4]:
print(list(KPMP_SN_raw.obs.columns))
print(list(KPMP_SN_raw.obs['cell_type'].unique()))
print(list(KPMP_SN_raw.obs['tissue'].unique()))
print(list(KPMP_SN_raw.obs['tissue_ontology_term_id'].unique()))

['library_id', 'nCount_RNA', 'nFeature_RNA', 'percent.er', 'percent.mt', 'subclass.l2', 'subclass.l1', 'class', 'experiment_id', 'suspension_type', 'assay_ontology_term_id', 'organism_ontology_term_id', 'donor_id', 'specimen', 'disease_category', 'disease_ontology_term_id', 'eGFR', 'diabetes_history', 'hypertension', 'sex_ontology_term_id', 'development_stage_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'is_primary_data', 'region', 'percent.cortex', 'percent.medulla', 'tissue_type', 'tissue_ontology_term_id', 'cell_type_ontology_term_id', 'Age_binned', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid']
['kidney interstitial alternatively activated macrophage', 'kidney distal convoluted tubule epithelial cell', 'epithelial cell of proximal tubule', 'T cell', 'kidney loop of Henle thick ascending limb epithelial cell', 'kidney collecting duct intercalated cell', 'kidney interstitial fibrobl

In [5]:
print(list(HuBMAP_RK_raw.obs.columns))
print(list(HuBMAP_RK_raw.obs['predicted_label'].unique()))
print(list(HuBMAP_RK_raw.obs['azimuth_label'].unique()))

['uuid', 'hubmap_id', 'age', 'sex', 'height', 'weight', 'bmi', 'cause_of_death', 'race', 'barcode', 'dataset', 'azimuth_label', 'azimuth_id', 'predicted_CLID', 'predicted_label', 'cl_match_type', 'prediction_score']
['kidney cortex collecting duct principal cell', 'kidney connecting tubule epithelial cell', 'vasa recta descending limb cell', 'kidney inner medulla collecting duct principal cell', 'papillary tips cell', 'renal medullary fibroblast', 'parietal epithelial cell', 'vasa recta ascending limb cell', 'kidney loop of Henle short descending thin limb epithelial cell', 'kidney loop of Henle thin ascending limb epithelial cell', 'kidney interstitial fibroblast', 'kidney resident macrophage', 'kidney loop of Henle long descending thin limb inner medulla epithelial cell', 'kidney loop of Henle medullary thick ascending limb epithelial cell', 'renal interstitial pericyte', 'epithelial cell of proximal tubule segment 3', 'mesangial cell', 'glomerular capillary endothelial cell', 'epith

In [20]:
KPMP_SN_raw.var.columns

Index(['feature_name', 'feature_reference', 'feature_biotype',
       'feature_length'],
      dtype='object')

In [5]:
"ENSG00000272954.1" in KPMP_SN_raw.var["feature_name"].astype(str).values


True

In [6]:
KPMP_shared = {
    "library_id": "dataset_id",
    "tissue_ontology_term_id": "as_id",
    "cell_type_ontology_term_id": "cl_id",
    "cell_type": "cl_label",
    "nCount_RNA": "gene_count",
    "Age_binned": "age",
    "sex": "sex",
    "self_reported_ethnicity": "race",
    "disease": "disease",
}
KPMP_SN_raw.obs['consortium'] = 'KPMP'
KPMP_SN_raw.obs['collection'] = 'KPMP SC RNAseq'

def filter_to_dataset_cell_types(
    adata,
    dataset_names,
    source_col='cell_type'
):
    # count before
    before = adata.n_obs

    # filter mask
    mask = adata.obs[source_col].isin(dataset_names)
    adata_filtered = adata[mask, :].copy()

    # report
    after = adata_filtered.n_obs
    print(
        f"Filtered to dataset cell types ({source_col}): "
        f"kept {after}/{before} cells ({before - after} removed)."
    )

    return adata_filtered

mapping = {
    # exact matches
    'B cell': 'B cell',
    'T cell': 'T cell',
    'conventional dendritic cell': 'conventional dendritic cell',
    'mast cell': 'mast cell',
    'mature NK T cell': 'mature NK T cell',
    'mononuclear phagocyte': 'mononuclear phagocyte',
    'neutrophil': 'neutrophil',
    'non-classical monocyte': 'non-classical monocyte',
    'papillary tips cell': 'papillary tips cell',
    'parietal epithelial cell': 'parietal epithelial cell',
    'plasma cell': 'plasma cell',
    'plasmacytoid dendritic cell, human': 'plasmacytoid dendritic cell, human',
    'podocyte': 'podocyte',
    'kidney interstitial fibroblast': 'kidney interstitial fibroblast',
    'renal interstitial pericyte': 'renal interstitial pericyte',
    # proximal tubule
    'epithelial cell of proximal tubule segment 1': 'epithelial cell of proximal tubule',
    'epithelial cell of proximal tubule segment 2': 'epithelial cell of proximal tubule',
    'epithelial cell of proximal tubule segment 3': 'epithelial cell of proximal tubule',
    # distal convoluted tubule
    'epithelial cell of early distal convoluted tubule': 'kidney distal convoluted tubule epithelial cell',
    'epithelial cell of late distal convoluted tubule': 'kidney distal convoluted tubule epithelial cell',
    # connecting tubule & collecting duct
    'kidney connecting tubule alpha-intercalated cell': 'kidney collecting duct intercalated cell',
    'kidney connecting tubule principal cell': 'kidney collecting duct principal cell',
    'kidney cortex collecting duct intercalated cell': 'kidney collecting duct intercalated cell',
    'kidney outer medulla collecting duct intercalated cell': 'kidney collecting duct intercalated cell',
    'kidney cortex collecting duct principal cell': 'kidney collecting duct principal cell',
    'kidney inner medulla collecting duct principal cell': 'kidney collecting duct principal cell',
    'kidney outer medulla collecting duct principal cell': 'kidney collecting duct principal cell',
    'kidney collecting duct intercalated cell': 'kidney collecting duct intercalated cell',
    'kidney collecting duct principal cell': 'kidney collecting duct principal cell',
    'kidney connecting tubule epithelial cell': 'kidney connecting tubule epithelial cell',
    # loop of Henle
    'kidney loop of Henle cortical thick ascending limb epithelial cell': 
        'kidney loop of Henle thick ascending limb epithelial cell',
    'kidney loop of Henle medullary thick ascending limb epithelial cell': 
        'kidney loop of Henle thick ascending limb epithelial cell',
    'kidney loop of Henle thick ascending limb epithelial cell': 
        'kidney loop of Henle thick ascending limb epithelial cell',
    'kidney loop of Henle thin ascending limb epithelial cell': 
        'kidney loop of Henle thin ascending limb epithelial cell',
    'kidney loop of Henle short descending thin limb epithelial cell': 
        'kidney loop of Henle thin descending limb epithelial cell',
    'kidney loop of Henle long descending thin limb inner medulla epithelial cell': 
        'kidney loop of Henle thin descending limb epithelial cell',
    'kidney loop of Henle long descending thin limb outer medulla epithelial cell': 
        'kidney loop of Henle thin descending limb epithelial cell',
    # endothelial
    'endothelial cell': 'endothelial cell',
}
HuBMAP_LK_raw = filter_to_dataset_cell_types(HuBMAP_LK_raw, mapping, source_col='predicted_label')


HuBMAP_RK_raw = filter_to_dataset_cell_types(HuBMAP_RK_raw, mapping, source_col='predicted_label')



def remove_slide_seq_cells(adata, slide_seq_ids):

    # Count before filtering
    total_before = adata.n_obs
    
    # Mask of cells to remove
    remove_mask = adata.obs['hubmap_id'].isin(slide_seq_ids)
    
    # Subset to keep only non-slide_seq cells
    adata_filtered = adata[~remove_mask, :].copy()
    total_after = adata_filtered.n_obs
    
    # Report
    print("Removing slide_seq cells only:")
    print(f"  Cells before: {total_before}")
    print(f"  Cells after:  {total_after}")
    print(f"  Cells removed: {total_before - total_after}")
    return adata_filtered
need_to_removed_slide_seq_HuBMAP=['HBM222.VQSW.335',
 'HBM248.HPXX.584',
 'HBM269.GDLH.894',
 'HBM294.XZLM.256',
 'HBM356.MDPN.792',
 'HBM363.FVKP.935',
 'HBM398.BLRW.228',
 'HBM456.CGDP.395',
 'HBM462.XQCR.933',
 'HBM522.QXVG.468',
 'HBM528.KNCB.488',
 'HBM547.SJSK.268',
 'HBM547.TFRR.794',
 'HBM595.LBXP.486',
 'HBM679.RLJH.432',
 'HBM735.FSBZ.626',
 'HBM757.KLKW.524',
 'HBM775.CMGG.464',
 'HBM785.XFTT.663',
 'HBM834.SLQN.292',
 'HBM883.PHQS.523',
 'HBM976.*','HBM232.MBNR.586', 'HBM266.FTJN.632', 'HBM297.FDTX.382', 'HBM445.HBRO.488',
 'HBM459.KCST.593', 'HBM532.KKRC.477', 'HBM634.JHVB.286', 'HBM634.ZSHF.736',
 'HBM647.QDBG.936', 'HBM736.MNMD.453', 'HBM827.MJMM.447', 'HBM846.KVCF.674',
 'HBM892.CCDZ.345', 'HBM965.PSNC.855', 'HBM986.KFWG.239', 'HBM232.MBNR.586']
HuBMAP_LK_raw=remove_slide_seq_cells(HuBMAP_LK_raw,need_to_removed_slide_seq_HuBMAP)
HuBMAP_RK_raw=remove_slide_seq_cells(HuBMAP_RK_raw,need_to_removed_slide_seq_HuBMAP)


KPMP_SN_raw.obs.rename(columns=KPMP_shared, inplace=True)

KPMP_var_shared = {
    'feature_name': 'gene_name'
}
KPMP_SN_raw.var.rename(columns=KPMP_var_shared, inplace=True)

Hubmap_shared = {
    "cell_id": "cell_id", 
    "predicted_CLID": "cl_id",
    "predicted_label": "cl_label",
    "n_genes": "gene_count",
    "age": "age",
    "sex": "sex",
    "race": "race",
    "hubmap_id":"dataset_id"
}
HubMap_var_shared = {
    "hugo_symbol": "gene_name"
}

def add_common_value(adata,RK):
 
    adata.obs['as_id'] = "UBERON:0002113"
    adata.obs['disease'] = "normal"
    adata.obs['consortium'] = 'HuBMAP'
    if RK == True:
        adata.obs['collection'] = 'HuBMAP Right Kidney'
    else:
        adata.obs['collection'] = 'HuBMAP Left Kidney'
    adata.obs['tissue'] = 'kidney'
    # Rename HuBMAP columns using inplace renaming
    adata.obs.rename(columns=Hubmap_shared, inplace=True)
    adata.var.rename(columns=HubMap_var_shared, inplace=True)
    adata.var.index = adata.var.index.to_series().apply(lambda x: re.sub(r'\.\d+$', '', x))
    return adata

HuBMAP_LK_raw = add_common_value(HuBMAP_LK_raw,False)
HuBMAP_RK_raw = add_common_value(HuBMAP_RK_raw,True)


def  remove_extra_var(adata):
    columns_to_drop = [col for col in adata.var.columns if col != 'gene_name']
    adata.var.drop(columns=columns_to_drop, inplace=True)
    return adata

KPMP_SN_raw=remove_extra_var(KPMP_SN_raw)

def drop_unqiue_obs_columns(adata):
    keep_cols = [
    'consortium',
    'collection',
    'dataset_id',
    'as_id',
    'cl_id',
    'cl_label',
    # 'gene_count',
    'age',
    'sex',
    'race',
    'disease'
    ]
    missing_cols = [col for col in keep_cols if col not in adata.obs.columns]
    if missing_cols:
        print("Warning: The following columns are missing in adata.obs:", missing_cols)
    available_cols = [col for col in keep_cols if col in adata.obs.columns]
    adata.obs = adata.obs[available_cols]
    return adata

HuBMAP_LK_raw = drop_unqiue_obs_columns(HuBMAP_LK_raw)
HuBMAP_RK_raw = drop_unqiue_obs_columns(HuBMAP_RK_raw)

# HuBMAP_combined=drop_unqiue_obs_columns(HuBMAP_combined)

KPMP_SN_raw=drop_unqiue_obs_columns(KPMP_SN_raw)

def map_the_gene_name(adata):
    common_genes = KPMP_SN_raw.var.index.intersection(adata.var.index)
    
    gene_names = KPMP_SN_raw.var.loc[common_genes, 'gene_name'].astype(str)
    
    adata.var['gene_name'] = adata.var.get('gene_name', pd.Series(dtype=str)).astype(str)
    
    adata.var.loc[common_genes, 'gene_name'] = gene_names
    
    print(f"Identified and updated {len(common_genes)} common gene IDs.")
    
    
    return adata

HuBMAP_LK_raw = map_the_gene_name(HuBMAP_LK_raw)
HuBMAP_RK_raw = map_the_gene_name(HuBMAP_RK_raw)


shared = (
    KPMP_SN_raw.var.index
    .intersection(HuBMAP_LK_raw.var.index)
    .intersection(HuBMAP_RK_raw.var.index)
)

print(f"Found {len(shared)} genes shared across all datasets.")

KPMP_SN_raw     = KPMP_SN_raw[:, shared].copy()
HuBMAP_LK_raw   = HuBMAP_LK_raw[:, shared].copy()
HuBMAP_RK_raw   = HuBMAP_RK_raw[:, shared].copy()
print("Obs columns before cocnat:", KPMP_SN_raw.obs.columns.tolist())

print("Obs columns before cocnat:", HuBMAP_LK_raw.obs.columns.tolist())

print("Obs columns before cocnat:", HuBMAP_RK_raw.obs.columns.tolist())
adata_concat_scvi = anndata.concat([KPMP_SN_raw,HuBMAP_LK_raw,HuBMAP_RK_raw], label="batch", keys=["KPMP", "HuBMAP_LK", "HuBMAP_RK"],axis=0, join="inner", merge="first")



Filtered to dataset cell types (predicted_label): kept 1371041/1522897 cells (151856 removed).
Filtered to dataset cell types (predicted_label): kept 1673578/1803736 cells (130158 removed).
Removing slide_seq cells only:
  Cells before: 1371041
  Cells after:  960932
  Cells removed: 410109
Removing slide_seq cells only:
  Cells before: 1673578
  Cells after:  1283320
  Cells removed: 390258
Identified and updated 36297 common gene IDs.
Identified and updated 36297 common gene IDs.
Found 36297 genes shared across all datasets.
Obs columns before cocnat: ['consortium', 'collection', 'dataset_id', 'as_id', 'cl_id', 'cl_label', 'age', 'sex', 'race', 'disease']
Obs columns before cocnat: ['consortium', 'collection', 'dataset_id', 'as_id', 'cl_id', 'cl_label', 'age', 'sex', 'race', 'disease']
Obs columns before cocnat: ['consortium', 'collection', 'dataset_id', 'as_id', 'cl_id', 'cl_label', 'age', 'sex', 'race', 'disease']


In [7]:
adata_concat_scvi_1=adata_concat_scvi.copy()
sc.pp.filter_cells(adata_concat_scvi, min_genes=200) 
sc.pp.filter_genes (adata_concat_scvi, min_cells=3) 


In [7]:
print(adata_concat_scvi.X)

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 1009461054 stored elements and shape (832343, 35878)>
  Coords	Values
  (0, 44)	1.0
  (0, 74)	1.0
  (0, 678)	1.0
  (0, 1638)	1.0
  (0, 1949)	1.0
  (0, 2157)	1.0
  (0, 3179)	1.0
  (0, 3226)	1.0
  (0, 3594)	1.0
  (0, 4418)	1.0
  (0, 4873)	1.0
  (0, 5484)	1.0
  (0, 6447)	1.0
  (0, 6495)	1.0
  (0, 6581)	5.0
  (0, 6612)	1.0
  (0, 7172)	1.0
  (0, 7236)	1.0
  (0, 7714)	1.0
  (0, 7716)	1.0
  (0, 8081)	1.0
  (0, 8113)	1.0
  (0, 8120)	1.0
  (0, 8170)	3.0
  (0, 8206)	1.0
  :	:
  (832342, 35631)	1.0
  (832342, 35641)	0.5
  (832342, 35670)	1.0
  (832342, 35681)	0.25
  (832342, 35682)	1.0
  (832342, 35696)	0.33333298563957214
  (832342, 35714)	0.5
  (832342, 35741)	1.0
  (832342, 35755)	0.5
  (832342, 35782)	1.0
  (832342, 35785)	1.0
  (832342, 35786)	0.5
  (832342, 35796)	0.5
  (832342, 35808)	0.5
  (832342, 35809)	0.5
  (832342, 35811)	1.0
  (832342, 35812)	1.0
  (832342, 35813)	1.0
  (832342, 35830)	0.5
  (832342, 35835)	0.5
  (832342,

In [8]:
sc.pp.normalize_total(adata_concat_scvi,target_sum=1e4)
sc.pp.log1p(adata_concat_scvi)

In [9]:
label_mapping = {
    # Proximal tubule
    "epithelial cell of proximal tubule": "Proximal tubule epithelial cell",
    "epithelial cell of proximal tubule segment 1": "Proximal tubule epithelial cell",
    "epithelial cell of proximal tubule segment 2": "Proximal tubule epithelial cell",
    "epithelial cell of proximal tubule segment 3": "Proximal tubule epithelial cell",

    # Distal convoluted tubule
    "kidney distal convoluted tubule epithelial cell": "Distal convoluted tubule epithelial cell",
    "epithelial cell of early distal convoluted tubule": "Distal convoluted tubule epithelial cell",
    "epithelial cell of late distal convoluted tubule": "Distal convoluted tubule epithelial cell",

    # Loop of Henle
    "kidney loop of Henle thin descending limb epithelial cell": "Loop of Henle epithelial cell ",
    "kidney loop of Henle thin ascending limb epithelial cell": "Loop of Henle epithelial cell ",
    "kidney loop of Henle cortical thick ascending limb epithelial cell": "Loop of Henle epithelial cell ",
    "kidney loop of Henle medullary thick ascending limb epithelial cell": "Loop of Henle epithelial cell ",
    "kidney loop of Henle long descending thin limb outer medulla epithelial cell": "Loop of Henle epithelial cell ",
    "kidney loop of Henle long descending thin limb inner medulla epithelial cell": "Loop of Henle epithelial cell ",
    "kidney loop of Henle short descending thin limb epithelial cell": "Loop of Henle epithelial cell ",

    # Collecting duct principal cells
    "kidney collecting duct principal cell": "Collecting duct principal cell",
    "kidney cortex collecting duct principal cell": "Collecting duct principal cell",
    "kidney outer medulla collecting duct principal cell": "Collecting duct principal cell",
    "kidney inner medulla collecting duct principal cell": "Collecting duct principal cell",

    # Collecting duct intercalated cells
    "kidney collecting duct intercalated cell": "Collecting duct intercalated cell",
    "kidney cortex collecting duct intercalated cell": "Collecting duct intercalated cell",
    "kidney outer medulla collecting duct intercalated cell": "Collecting duct intercalated cell",

    # Connecting tubule
    "kidney connecting tubule epithelial cell": "Connecting tubule epithelial cell",
    "kidney connecting tubule principal cell": "Connecting tubule epithelial cell",
    "kidney connecting tubule alpha-intercalated cell": "Connecting tubule epithelial cell",

    # Other epithelial
    "podocyte": "Podocyte",
    "parietal epithelial cell": "Parietal epithelial cell",
    "papillary tips cell": "Papillary tips cell",

    # Stromal & endothelial
    "endothelial cell": "Endothelial cell",
    "kidney interstitial cell": "Kidney interstitial cell",
    "kidney interstitial fibroblast": "Kidney interstitial fibroblast",
    "kidney interstitial alternatively activated macrophage": "Kidney interstitial alternatively activated macrophage",
    "renal interstitial pericyte": "Renal interstitial pericyte",

    # Immune cells
    "T cell": "T cell",
    "cytotoxic T cell": "T cell",
    "mature NK T cell": "T cell",
    "B cell": "B cell",
    "plasma cell": "Plasma cell",
    "natural killer cell": "Natural killer cell",
    "monocyte": "Monocyte",
    "non-classical monocyte": "Monocyte",
    "conventional dendritic cell": "Dendritic cell",
    "plasmacytoid dendritic cell, human": "Dendritic cell",
    "mononuclear phagocyte": "Mononuclear phagocyte",
    "mast cell": "Mast cell",
    "neutrophil": "Neutrophil",
}

adata_concat_scvi.obs["general_cl_label"] = adata_concat_scvi.obs["cl_label"].map(label_mapping)

adata_concat_scvi.obs["general_cl_label"] = adata_concat_scvi.obs["general_cl_label"].fillna(adata_concat_scvi.obs["cl_label"])
adata_concat_scvi.obs['age'] = adata_concat_scvi.obs['age'].astype(str)

print(adata_concat_scvi.obs[["cl_label", "general_cl_label"]].head(20))

                                                                  cl_label  \
KC33_AAACCCAAGTAGTCAA-1  kidney interstitial alternatively activated ma...   
KC33_AAACCCACAACTGAAA-1    kidney distal convoluted tubule epithelial cell   
KC33_AAACCCACACGTTGGC-1                 epithelial cell of proximal tubule   
KC33_AAACCCAGTACATACC-1                                             T cell   
KC33_AAACCCATCCGAGCTG-1                 epithelial cell of proximal tubule   
KC33_AAACCCATCTTCGTGC-1                 epithelial cell of proximal tubule   
KC33_AAACGAACAACGATCT-1  kidney loop of Henle thick ascending limb epit...   
KC33_AAACGAACACCGTGGT-1  kidney loop of Henle thick ascending limb epit...   
KC33_AAACGAACAGTAGAAT-1                                             T cell   
KC33_AAACGAACAGTATACC-1                 epithelial cell of proximal tubule   
KC33_AAACGAAGTAGGCAGT-1                 epithelial cell of proximal tubule   
KC33_AAACGAAGTGCATGAG-1                 epithelial cell of proxi

In [10]:
adata_concat_scvi.obs["general_cl_label"].unique()

array(['Kidney interstitial alternatively activated macrophage',
       'Distal convoluted tubule epithelial cell',
       'Proximal tubule epithelial cell', 'T cell',
       'kidney loop of Henle thick ascending limb epithelial cell',
       'Collecting duct intercalated cell',
       'Kidney interstitial fibroblast',
       'Connecting tubule epithelial cell',
       'Collecting duct principal cell', 'Endothelial cell', 'B cell',
       'Mononuclear phagocyte', 'Podocyte', 'Parietal epithelial cell',
       'Mast cell', 'Dendritic cell', 'Plasma cell', 'Monocyte',
       'Loop of Henle epithelial cell ', 'Renal interstitial pericyte',
       'Neutrophil', 'Papillary tips cell', 'neural cell'], dtype=object)

In [11]:
print(f"Total Cells: {adata_concat_scvi.n_obs:,}")
print(f"Total Genes: {adata_concat_scvi.n_vars:,}")  
adata_concat_scvi.write_h5ad("/scratch/yongqunh_root/yongqunh0/wruopeng/HuBMAP_KPMP/kpmp_hubmap_combined_after_1e4_and_log_normalization_afterQC.h5ad")


Total Cells: 832,343
Total Genes: 35,878


In [12]:
new_combined_not_log=sc.read("/scratch/yongqunh_root/yongqunh0/wruopeng/HuBMAP_KPMP/kpmp_hubmap_combined_after_1e4_and_log_normalization_afterQC.h5ad")

In [13]:

# Run t-test markers
sc.tl.rank_genes_groups(
    new_combined_not_log,
    groupby="general_cl_label",
    method="t-test",
    n_genes=20,
    use_raw=False,
    tie_correct=False,
    key_added="t_test_markers",
)



/home/wruopeng/.local/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:435: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, "names"] = self.var_names[global_indices]
/home/wruopeng/.local/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:437: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, "scores"] = scores[global_indices]
/home/wruopeng/.local/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:440: PerformanceWarning: DataFrame is highly fragme

In [14]:
import scipy.sparse as sp

In [15]:
# Get the tidy table (names, scores, logfoldchanges, pvals, pvals_adj, group)
df_markers_non_log = sc.get.rank_genes_groups_df(
    new_combined_not_log, group=None, key="t_test_markers"
)

# ---- Add means & means_rest from .uns ----
uns = new_combined_not_log.uns["t_test_markers"]


In [16]:

df_markers_non_log = sc.get.rank_genes_groups_df(
    new_combined_not_log,
    group=None,
    key="t_test_markers"
)

In [ ]:


def attach_means_from_matrix(adata, df, group_key="general_cl_label", layer=None, round_ndigits=6):
    X = adata.layers[layer] if layer else adata.X
    is_sparse = sp.issparse(X)
    if is_sparse:
        X = X.tocsr()

    var_names = adata.var_names
    name_to_idx = {name: i for i, name in enumerate(var_names)}

    groups = adata.obs[group_key].astype("category")
    cats = list(groups.cat.categories)
    masks = {c: (groups.values == c) for c in cats}

    def _mean_for_mask(col_idx, mask_bool):
        n = int(mask_bool.sum())
        if n == 0:
            return np.nan
        if is_sparse:
            s = X[mask_bool, col_idx].sum()
            s = float(s) if np.isscalar(s) else float(np.asarray(s).ravel()[0])
            return s / n
        else:
            return float(X[mask_bool, col_idx].mean())

    pairs = df[["group", "names"]].drop_duplicates()
    rows = []
    for g, gene in pairs.itertuples(index=False):
        print(gene)
        j = name_to_idx.get(str(gene))
        if j is None:
            rows.append({"group": g, "names": gene, "means": np.nan, "means_rest": np.nan})
            continue
        mask_in = masks[g]
        mean_in = _mean_for_mask(j, mask_in)

        if round_ndigits is not None:
            if np.isfinite(mean_in):  mean_in = round(mean_in, round_ndigits)

        rows.append({"group": g, "names": gene, "means": mean_in})

    df_means = pd.DataFrame(rows)
    return df.merge(df_means, on=["group", "names"], how="left")

df_markers_non_log = attach_means_from_matrix(
    new_combined_not_log, df_markers_non_log,
    group_key="general_cl_label", layer=None, round_ndigits=6
)


ENSG00000144218
ENSG00000166501
ENSG00000153064
ENSG00000075884
ENSG00000111913
ENSG00000065413
ENSG00000168918
ENSG00000116191
ENSG00000081237
ENSG00000183049
ENSG00000136573
ENSG00000137478
ENSG00000112576
ENSG00000156738
ENSG00000134516
ENSG00000185811
ENSG00000048740
ENSG00000138639
ENSG00000155849
ENSG00000196092
ENSG00000147606
ENSG00000147614
ENSG00000109684
ENSG00000105929
ENSG00000143882
ENSG00000069122
ENSG00000151276
ENSG00000115112
ENSG00000154678
ENSG00000172348
ENSG00000146376
ENSG00000197943
ENSG00000074410
ENSG00000249601
ENSG00000214128
ENSG00000172164
ENSG00000049759
ENSG00000113073
ENSG00000112530
ENSG00000139173
ENSG00000167580
ENSG00000150201
ENSG00000107485
ENSG00000165272
ENSG00000171435
ENSG00000135374
ENSG00000111371
ENSG00000017797
ENSG00000173898
ENSG00000166828
ENSG00000112319
ENSG00000182752
ENSG00000120885
ENSG00000251562
ENSG00000159388
ENSG00000138696
ENSG00000171345
ENSG00000163435
ENSG00000119314
ENSG00000187889
ENSG00000183023
ENSG00000136531
ENSG0000

In [18]:
# 4) Save
df_markers_non_log.to_csv("markers_ttest_scanpy_log_mean_value.csv", index=False)